# Load data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Set up the MTP and train

In [2]:
from mlearn.potentials.mtp import MTPotential

mtp = MTPotential()
mtp.train(train_structures=train_structures, energies=train_energies, forces=train_forces, unfitted_mtp='08g.mtp', 
          energy_weight=1, force_weight=0.01, stress_weight=0, max_dist=4.8, max_iter=500)

0

# Lattice constant, Elastic constant

In [3]:
from mlearn.potentials.lammps.calcs import LatticeConstant

conventional_cell = Structure.from_file('conventional.cif')
lc_calculator = LatticeConstant(ff_settings=mtp)
a, b, c = lc_calculator.calculate([conventional_cell])[0]
print('Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Lattice a: 3.36157650899549, Lattice b: 3.36157650899548, Lattice c: 3.36157650899549


In [4]:
from mlearn.potentials.lammps.calcs import ElasticConstant

ec_calculator = ElasticConstant(ff_settings=mtp, lattice='bcc', alat=3.106)
C11, C12, C44, bulk_modulus = ec_calculator.calculate()
print('C11: {}, C12: {}, C44: {}, bulk modulus: {}'.format(C11, C12, C44, bulk_modulus))

C11: 325.163807449281, C12: 134.403380849836, C44: 98.8407448572887, bulk modulus: 197.990189655684


# Load model from *.mtp

In [5]:
mtp_loaded = MTPotential.from_config('fitted.mtp')

# Energy, force, stress prediction

In [6]:
from mlearn.potentials.lammps.calcs import EnergyForceStress

struct = Structure.from_file('test_struct.cif')
efs_calculator = EnergyForceStress(ff_settings=mtp_loaded)
energy, forces, stresses = efs_calculator.calculate([struct])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

energy: -517.912347611787
forces: 
 [[-0.567287  -0.999218   2.99811  ]
 [-2.93841    0.360879  -0.264224 ]
 [ 1.9756    -4.50984    2.50153  ]
 [ 1.72667    0.596511  -0.163787 ]
 [ 0.632484   1.25383   -0.303409 ]
 [ 1.59355   -1.6859     0.430083 ]
 [ 2.54958   -3.57603   -1.18008  ]
 [-0.737123  -0.947363  -0.45699  ]
 [-2.00566    1.60794   -0.399287 ]
 [-2.56704    3.22097    2.65476  ]
 [ 3.63995   -2.04146    3.05754  ]
 [ 3.27543    3.93896   -1.72776  ]
 [ 0.311548   1.10147    0.546629 ]
 [-2.47146    1.37583    1.23884  ]
 [-2.43209    1.06086    0.875812 ]
 [-2.75257   -0.160059   1.46723  ]
 [-0.79132   -2.998      0.175696 ]
 [ 1.02875   -1.70727    0.440536 ]
 [-0.498761   0.581282  -0.409875 ]
 [-2.83857    0.263698   2.05643  ]
 [-0.598159  -3.46901    1.02386  ]
 [-4.60597   -0.997348  -0.524876 ]
 [ 0.730853  -0.596296  -2.21453  ]
 [ 1.29994   -1.29897    0.0757194]
 [ 2.15297   -0.0159131 -1.71414  ]
 [-3.30582   -2.67634    0.978358 ]
 [-5.91066   -2.07501    0.4